<h1> 2c. バッチの追加と特徴量の作成によるリファクタリング</h1>

このnotebookでは、同じく小さなデータセットを読み込みますが、機械学習のパイプラインを2つの重要な点でリファクタリングします。

<ol>
<li> 入力データをバッチに分けて読み込むようにリファクタリングする
<li> 入力と一対一対応とならないように、特徴量の作成をする
</ol>

以前のnotebookで扱ったPandasの関数も、すべてのデータをメモリ上に読み込んだあとでバッチに分けています。しかし巨大なデータセットに対してこれを適用することはできません。


In [ ]:
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

<h2> 1. 入力をリファクタリングする </h2>

Lab 1aで作成したデータを読み込みますが、今回はより一般的でパフォーマンスの良い方法で行います。<br>
Pandasを利用せずに、TensorFlowのDataset APIを利用します。

In [ ]:
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], [-74.0], [40.0], [-74.0], [40.7], [1.0], ['nokey']]

def read_dataset(filename, mode, batch_size = 512):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
      label = features.pop(LABEL_COLUMN)
      return features, label

    # Create list of files that match pattern
    file_list = tf.gfile.Glob(filename)

    # Create dataset from file list
    dataset = tf.data.TextLineDataset(file_list).map(decode_csv)
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)
    else:
        num_epochs = 1 # end-of-input after this

    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()
  return _input_fn
    

def get_train():
  return read_dataset('./taxi-train.csv', mode = tf.estimator.ModeKeys.TRAIN)

def get_valid():
  return read_dataset('./taxi-valid.csv', mode = tf.estimator.ModeKeys.EVAL)

def get_test():
  return read_dataset('./taxi-test.csv', mode = tf.estimator.ModeKeys.EVAL)

<h2> 2.  特徴量の作成方法をリファクタリングする </h2>

ここでは、以下は空のままでスルーします。しかし、このこのようにリファクタリングを行うことで、入力と特徴量の一対一の関係が崩されることになります。

In [ ]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('pickuplon'),
    tf.feature_column.numeric_column('pickuplat'),
    tf.feature_column.numeric_column('dropofflat'),
    tf.feature_column.numeric_column('dropofflon'),
    tf.feature_column.numeric_column('passengers'),
]

def add_more_features(feats):
  # Nothing to add (yet!)
  return feats

feature_cols = add_more_features(INPUT_COLUMNS)

<h2> モデルを作成し学習する</h2>

num_steps * batch_sizeのデータ数に対して学習を行っていることに注目してください。

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.LinearRegressor(
      feature_columns = feature_cols, model_dir = OUTDIR)
model.train(input_fn = get_train(), steps = 100);  # TODO: change the name of input_fn as needed

<h3> モデルを評価する</h3>

前回と同様に、検証データを用いて評価をします。3つ目のリファクタリング（評価を学習ループの中に入れる）については次のラボで扱います。

In [ ]:
def print_rmse(model, name, input_fn):
  metrics = model.evaluate(input_fn = input_fn, steps = 1)
  print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
print_rmse(model, 'validation', get_valid())

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License